# Pronalaženje gena
CRF model za pronalaženje gena na osnovu modela: https://www.researchgate.net/figure/A-finite-state-machine-for-gene-prediction-gion-N-C-In-our-model-the-introns-and_fig2_221605559

In [1]:
from sklearn_crfsuite import CRF

In [2]:
'''Izdvajanje feature-a'''
def create_features(X):
    features = []
    stop_codons = ['tga', 'taa', 'tag']
    
    for i in range(len(X)):
        x = X[i]
        
        current_features = {
#             'bias': 1,
            'nuc': x.lower()
        }
        
        if i > 0:
            current_features.update({
                '-1:nuc': X[i - 1].lower(),
                'is_ag': X[i-1: i + 1].lower() == 'ag',
                'is_gt': X[i-1: i + 1].lower() == 'gt',
            })
            
        if i > 1:
            current_features.update({
                '-2:nuc': X[i - 2].lower(),
                'is_start': X[i-2: i + 1].lower() == 'atg',
                'is_stop': X[i-2: i + 1].lower() in stop_codons,
            })

        features.append(current_features)
    
    return features

## Učitavanje podataka o genima

In [3]:
from Bio import SeqIO, Seq

In [4]:
# Učitavanje CDS sekvenci kvasca
data = SeqIO.parse("data/NC_001146.8.gb", "genbank")

In [5]:
records = []
for record in data:
    records.append(record)
record = records[0]

In [6]:
# Izdvajanje introna (mala slova) i egzona (velika slova) iz genbank podataka

sequence = record.seq
CDS = [feature for feature in record.features if feature.type == 'CDS' and 'gene' in feature.qualifiers]

X_train = []

regex = r'(\[(\d+):(\d+)\]\(([\+\-])\))'
for cds_record in CDS:
    segments = ([(int(x._start), int(x._end), int(x._strand)) for x in cds_record.location.parts])
    
    min_start = min(map(lambda x: x[0], segments))
    max_end = max(map(lambda x: x[1], segments))
    
    seq_direction = segments[0][2]
    
    gene = ''
    gene = sequence[min_start: max_end]
        
    gene = list(gene.lower())
    
    for (start, end, direction) in segments:
        exon = sequence[start:end]
        
        gene[start - min_start: end - min_start] = list(exon.upper())
        
    gene_seq = Seq.Seq(''.join(gene))
        
    if seq_direction == -1:
        gene_seq = gene_seq.reverse_complement()
    
    X_train.append(str(gene_seq))

In [7]:
# Izdvajanje sekvenci koje sadrže i introne i egzone

def has_lowercase(x):
    for c in x:
        if c.islower():
            return True
    return False
        
X_train_filtered = [x for x in X_train if has_lowercase(x)]

In [8]:
from random import choice

# Anotacija stanja po pozicijama u sekvenci
def prepare_train_set(X_train):
    X_copy = X_train[:]
    n = len(X_copy)
    y_train = []
    
    for i in range(n):
        x = X_copy[i]
        
        prefix_length = choice([i for i in range(0, 100, 3)])
        suffix_length = choice([i for i in range(0, 100, 3)])
        
        prefix = ''.join([choice(['A','T','C','G']) for _ in range(prefix_length)])
        suffix = ''.join([choice(['A','T','C','G']) for _ in range(suffix_length)])
        
        y_current = ['N' for _ in range(prefix_length)] + ['START0','START1','START2']
        
        
        in_intron = False
        m = len(x)
        k = 0
        
        ik = 0
        for j in range(3, m - 3):
            c = x[j]
            
            if c.islower():
                if not in_intron:
                    ik = 0
                    y_current.append('ISTART1')
                    in_intron = True
                
                elif ik == 0:
                    y_current.append('ISTART2')
                    ik = 1
                    
                else:
                    k = (k - 1) % 3                    
                    y_current.append(f'I{k}')
            else:
                if in_intron:
                    in_intron = False
                    y_current = y_current[:-2]
                    y_current.append('IEND1')
                    y_current.append('IEND2')                

                y_current.append(f'C{k % 3}')
                k = (k + 1) % 3
            
        y_current += ['STOP0','STOP1','STOP2'] + ['N' for _ in range(suffix_length)]
        X_copy[i] = (prefix + X_train[i] + suffix)
            
        y_train.append(y_current)
            
    return X_copy, y_train

In [9]:
X_prepared, y_train = prepare_train_set(X_train)

In [10]:
# Izdvajanje feature-a i deljenje na trening i test skupove
X_features = [create_features(x) for x in X_prepared]

X_train_features = X_features[:100]
y_train_subset = y_train[:100]

X_test_features = X_features[100:]
y_test_subset = y_train[100:]

In [11]:
crf = CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=100,
    all_possible_transitions=False
)

try:
    crf.fit(X_train_features, y_train_subset)
except AttributeError:
    pass

In [12]:
# Analiza predikcije po pozicijama
'''
k = 0
res = crf.predict([X_test_features[k]])[0]

for i in range(len(res)):
    print(X_prepared[100 + k][i], res[i], y_test_subset[k][i])
'''

'\nk = 0\nres = crf.predict([X_test_features[k]])[0]\n\nfor i in range(len(res)):\n    print(X_prepared[100 + k][i], res[i], y_test_subset[k][i])\n'

In [13]:
# Evaluacija modela
crf.score(X_test_features, y_test_subset)

0.9199970943882779

In [14]:
# Analiza težinskih faktora prelazaka između stanja
crf.transition_features_

{('N', 'N'): 10.013434,
 ('N', 'START0'): 5.264839,
 ('START0', 'START1'): 5.449518,
 ('START1', 'START2'): 3.512151,
 ('START2', 'C0'): 5.054681,
 ('C0', 'C1'): 11.525994,
 ('C0', 'ISTART1'): 6.060941,
 ('C0', 'STOP0'): 0.938522,
 ('C1', 'C2'): 11.502387,
 ('C1', 'ISTART1'): 4.718832,
 ('C1', 'STOP0'): 1.963676,
 ('C2', 'C0'): 11.428823,
 ('C2', 'ISTART1'): 4.423735,
 ('C2', 'STOP0'): 6.712074,
 ('ISTART1', 'ISTART2'): 0.677849,
 ('ISTART2', 'I0'): 0.268536,
 ('ISTART2', 'I2'): 0.111217,
 ('ISTART2', 'I1'): 0.038007,
 ('I0', 'I2'): 8.078463,
 ('I2', 'I1'): 8.059942,
 ('I2', 'IEND1'): 0.03795,
 ('I1', 'I0'): 8.024804,
 ('I1', 'IEND1'): 0.437598,
 ('IEND1', 'IEND2'): 0.699035,
 ('IEND2', 'C0'): 5.330731,
 ('IEND2', 'C2'): 6.358932,
 ('STOP0', 'STOP1'): 4.260613,
 ('STOP1', 'STOP2'): 4.273356,
 ('STOP2', 'N'): 4.589356}

In [15]:
# Analiza težinskih faktora feature-a u zavisnosti od stanja
for el in crf.state_features_.items():
    print(el)

(('nuc:a', 'N'), 1.085606)
(('nuc:a', 'START0'), 5.251026)
(('nuc:a', 'C0'), 0.544734)
(('nuc:a', 'C1'), 0.8979)
(('nuc:a', 'C2'), 0.911564)
(('nuc:a', 'I0'), 1.337723)
(('nuc:a', 'I2'), 2.056512)
(('nuc:a', 'I1'), 2.041313)
(('nuc:a', 'IEND1'), 4.155104)
(('nuc:a', 'STOP1'), 2.697266)
(('nuc:a', 'STOP2'), 2.755363)
(('nuc:g', 'N'), 1.489587)
(('nuc:g', 'START2'), 3.821629)
(('nuc:g', 'C0'), 0.811767)
(('nuc:g', 'C1'), 0.907766)
(('nuc:g', 'C2'), 0.78316)
(('nuc:g', 'ISTART1'), 4.334797)
(('nuc:g', 'I0'), 1.520007)
(('nuc:g', 'I2'), 1.991236)
(('nuc:g', 'I1'), 1.613535)
(('nuc:g', 'IEND2'), 3.896444)
(('nuc:g', 'STOP1'), 3.201275)
(('nuc:g', 'STOP2'), 1.805485)
(('-1:nuc:a', 'N'), 1.172655)
(('-1:nuc:a', 'START0'), 2.268955)
(('-1:nuc:a', 'START1'), 2.921926)
(('-1:nuc:a', 'C0'), 0.539556)
(('-1:nuc:a', 'C1'), 0.495406)
(('-1:nuc:a', 'C2'), 0.87104)
(('-1:nuc:a', 'ISTART1'), 1.130296)
(('-1:nuc:a', 'I0'), 1.354518)
(('-1:nuc:a', 'I2'), 2.214202)
(('-1:nuc:a', 'I1'), 2.135019)
(('-1:nuc